# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Aprendizaje no supervisado

### Trabajo práctico entregable - Grupo 22 - FIFA female players 2023 - Parte 2: implementación de modelos de ML

**Integrantes:**
- Chevallier-Boutell, Ignacio José
- Ribetto, Federico Daniel
- Rosa, Santiago
- Spano, Marcelo

**Seguimiento:** Meinardi, Vanesa

---

## Enunciados:

4- Aplicación de clustering para encontrar grupos de jugadoras con habilidades equivalentes, por ejemplo, jugadoras que podrían intercambiarse en el caso de una lesión o cuando una jugadora está cansada. Para esto utilice como mínimo dos técnicas de clustering: por ejemplo k-medias, DBSCAN, mezcla de Gaussianas y/o alguna jerárquica. Justifiquen por qué eligen los diferentes hiper-parámetros que se puedan elegir según el método: número de clusters, medida de distancia, criterio de aglomeración… 

5- Análisis cualitativo de los clusters encontrados. ¿Qué hay en cada cluster? ¿Son efectivamente equivalentes las jugadoras de un cluster, es decir, podrían cumplir el mismo rol en un equipo? Si se trata de clusters heterogéneos, ¿por qué razón pueden haber sido agrupadas las jugadoras del cluster? ¿Qué motiva las diferencias en tamaño?

6- Uso de alguna transformación (proyección, Embedding) para visualizar los resultados y/o usarla como preprocesado para aplicar alguna técnica de clustering.

## Librerías

Inicializamos el entorno.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from plots import plots

import matplotlib.cm as cm
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_samples, silhouette_score

pd.set_option('display.max_columns',150)
pd.set_option('display.max_rows',150)

## Lectura del dataset

Cargamos el conjunto de datos y lo procesamos para quedarnos sólo con lo que nos interesa. No usamos a las arqueras ya que son un cluster propio.

In [ ]:
path = 'fifa2023.csv'
fifa23 = pd.read_csv(path)
print(fifa23.keys())
fifa23_mod = fifa23[fifa23['gk']<50].copy()

vars_mod = ['crossing', 'finishing', 'heading', 'short_passing', 'volleys', 
            'marking', 'standing_tackle', 'sliding_tackle', 'acceleration', 
            'sprint', 'agility', 'balance', 'shot_power', 'stamina', 
            'long_shots', 'dribbling', 'curve', 'fk_acc', 'long_passing', 
            'ball_control', 'aggression', 'interceptions', 'positioning', 
            'vision', 'penalties', 'composure', 'ls', 'st', 'rs', 'lw', 'lf', 
            'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 
            'rm', 'ldm', 'cdm', 'rdm', 'lwb', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 
            'rb']

# No resetear índices, para que después sea fácil buscar nombre y demás
fifa23_mod = fifa23_mod[vars_mod]/100.

# Hierarchical Clustering

Un método de realizar clusterizaciones de puntos es construir una jerarquía de clusters. La estrategia puede ser **aglomerativa**, en la cual cada punto es un cluster distinto y en cada iteración se van agrupando, o **divisiva**, donde todos los puntos comienzan en el mismo cluster y se van dividiendo.

Para decidir cómo aglomerar o dividir los clusters, se utiliza como criterio la distancia entre puntos como métrica. El criterio y la definición de distancia varía de método a método. En particular, utilizamos la clusterización aglomerativa de scikit-learn `sklearn.cluster.AgglomerativeClustering` con la distancia default (euclidea), y el criterio de asignación default (minimización de los clusters aglomerados).

## Criterio del score de silueta

Para obtener el número óptimo de clústers, utilizamos el *score de silueta*.

Asumamos que tenemos nuestro conjunto de datos clusterizado. Sea $i \in C_{I}$ ($i$-esimo punto del cluster $C_I$), y sea

$$
    a(i) = \frac{1}{|C_I|-1}\sum_{j\in C_I, i \neq j} d(i,j)
$$

la distancia media entre $i$ y el resto de los puntos del cluster, donde $|C_I|$ es el número de puntos del clúster $C_I$ y $d(i,j)$ es la distancia entre los puntos $i$ y $j$.

Definimos la disimilaridad media $b(i)$ de un punto $i$ con un cluster $C_J$ como la distancia media entre el punto $i$ y todos los puntos en $C_J$ (con $C_J \neq C_I$).

Para cada punto $i \in C_I$, definimos 

$$
    b(i) = \min_{J \neq I} \frac{1}{|C_J|}\sum_{j\in C_J} d(i,j)
$$

como la distancia media más pequeña a todos los puntos en cualquier otro cluster (donde no pertenece $i$). El cluster con la disimilaridad más pequeña se denomina 'cluster vecino' de $i$. Definimos ahora la *silueta* de un punto $i$ como 


$$
    s(i) = \frac{b(i)-a(i)}{\max{a(i),b(i)}}
$$

coeficiente que toma valores en el intervalo $[-1,1]$. Si $s(i)$ está cerca de 1, el punto está apropiadamente clusterizado, mientras que si toma valores cercanos a -1, el punto tedría que estar asociado a su cluster vecino.

La media de $s(i)$ sobre todos los puntos $i$ es una medida que nos indica qué tan estrechamente agrupado está el clúster.


En esta nueva gráfica vemos que el cluster 0 (de carácter mediocampo/defensivo) y el cluster 2 (de carácter defensivo) se localizan en la región derecha, lo cual tiene sentido ya que en esa dirección juegan un rol importante las habilidades defensivas. Además, los clusters restantes (con naturalezas más ofensivas) de ubican del lado izquierdo, sentido en el que pesan más las habilidades ofensivas.

# Clustering

Empezamos cargando la base de datos y quedándonos con las variables que nos interesa, como vimos en la exploración de los datos.

Hagamos algunas clusterizaciones iniciales y midamos su efectividad utilizando el score de silueta, utilizando algunas posiciones para calcular dicha métrica.

Vemos que la efectividad de la clusterización depende de las variables con que medimos el score de silueta y del número de clusters.

In [ ]:
from random import sample

positions = ['ls', 'st', 'rs', 'lw', 'lf',
'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm',
'rm', 'ldm', 'cdm', 'rdm', 'lwb', 'rwb', 'lb', 'lcb', 'cb', 'rcb',
'rb']

npicks = 5

positions0 = sample(positions,npicks)
nvar = len(positions0)

#ploteo el score de silueta para algunas variables para tener una idea de la clusterización
for nclus in range(2,5):

    for i in range(npicks):
        for j in range(i):

            x = fifa23_mod[positions0[i]]
            y = fifa23_mod[positions0[j]]

            hierarchical_cluster = AgglomerativeClustering(n_clusters=nclus, metric='euclidean', linkage='ward')
            labels = hierarchical_cluster.fit_predict(fifa23_mod)

            # The silhouette_score gives the average value for all the samples.
            # This gives a perspective into the density and separation of the formed
            # clusters
            silhouette_avg = silhouette_score(np.array([x,y]).T, labels)
            # Compute the silhouette scores for each sample
            sample_silhouette_values = silhouette_samples(np.array([x,y]).T, labels)

            x_label = positions0[i]
            y_label = positions0[j]
            fig_name = 'tmp/results__nclus'+str(nclus)+'_comun_pos_'+positions0[i]+'_'+positions0[j]

            plots(x,y,x_label,y_label,labels,nclus,sample_silhouette_values,silhouette_avg,fig_name)


Para tener una idea más cuantitativa de la cantidad de clusters óptimos según el score de silueta, calculamos el score promedio para cada posición en función del número de clústers.

**warning**: esta celda tarda mucho en correr (aprox. una hora y media). 

**Spoiler alert**: el número óptimo de clusters es 3, se puede pasar directamente a las otras celdas.


In [ ]:
import os
try: os.mkdir('./tmp')
except: pass

n_max = 6
optimal_n_clus = []
for i in range(len(positions)):
    print('posición: '+positions[i]+', '+str(len(positions[:i])+1)+' de '+str(len(positions)))
    
    silhouette_avg_lst = []
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title('avg score of '+positions[i])
    ax.set_ylabel('silouette score')
    ax.set_xlabel('number ofclusters')

    x = fifa23_mod[positions[i]]
    for nclus in range(2,n_max+1):
        silhouette_avg=0
        for j in range(len(positions)):
            if i!=j:

                y = fifa23_mod[positions[j]]

                hierarchical_cluster = AgglomerativeClustering(n_clusters=nclus, metric='euclidean', linkage='ward')
                labels = hierarchical_cluster.fit_predict(fifa23_mod)
                silhouette_avg += silhouette_score(np.array([x,y]).T, labels)
                
        silhouette_avg_lst.append(silhouette_avg)
    silhouette_avg_lst = np.array(silhouette_avg_lst)/(len(positions)-1)
    optimal_n_clus.append(np.argmax(silhouette_avg_lst))

    ax.plot(np.linspace(2,n_max,n_max-1,dtype='int'), silhouette_avg_lst, linestyle = '--',linewidth=.7, marker='.',markersize=8)
    fig.savefig('tmp/avg_sil_score_'+str(positions[i]))

nclus_opt = int(round(np.mean(optimal_n_clus))+2,0) #redondeo el promedio

print('nro optimo de clusters:',nclus_opt)

Ahora que tenemos el número óptimo de clusters, veamos si la clusterización jerárquica realizada tiene sentido.

In [ ]:
nclus_opt = 3

hierarchical_cluster = AgglomerativeClustering(n_clusters=nclus_opt, metric='euclidean', linkage='ward')
labels = hierarchical_cluster.fit_predict(fifa23_mod)

df_clusters = fifa23_mod.copy()
df_clusters['ncluster'] = labels # Creamos una nueva columna con el cluster asignado
df_clusters.head()

Vemos que el cluster mayoritario contiene al 42% de las jugadoras, el segundo el 33%, y el minoritario el 25%.

In [ ]:
df_clusters.ncluster.value_counts(normalize=True)

En la siguiente imagen podemos ver cada una de las posiciones en la cancha:

![](plt/positions_fifa.jpg )

El primer cluster agrupa mayoritariamente a las mediocampistas (47% de los datos), y en menor medida, delanteras de los costados (21% de los datos).

El segundo cluster agrupa a las defensoras (74%) y, en menor medida, mediocampistas (21%).

El tercer cluster agrupa a las jugadoras ofensivas (87%).

Concluimos que la clusterización jerárquica logra obtener un agrupamiento esperable, obteniendo tres clusters donde divide a las jugadoras en defensoras, mediocampistas y delanteras.

In [ ]:
df_pos = df_clusters.copy()
df_pos['position'] = fifa23['position'] # Agrego las posisiones para filtrar

for n in range(nclus_opt):
    df_pos0 = df_pos[df_pos['ncluster']==n]
    print(df_pos0.position.value_counts(normalize=True))
